In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.spatial.distance as dist
%matplotlib inline
import pylab
import json
import random
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error as mse

from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#df_users = pd.read_json('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/user.json', lines=True)

In [3]:
#df_users.head()

# Reading big file line by line, taking first 10000 reviews

In [4]:
#c=0
#s = open('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/review_sub.json','w',encoding='utf8')
#with open('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/review.json', encoding='utf8') as f:
#    for line in f:
#        c=c+1
#        s.write(line)
#        if(c == 10000):
#            break

In [2]:
res_mer = pd.read_csv('D:\desk_2104\CS Sem4\CS6220\Project\dataset\yelp.train.rating', sep ='\t')

In [3]:
res_mer.head()

,user_id,business_id,rating,date
0,0,0,3,2015-03-19
1,0,1,3,2015-03-19
2,0,2,4,2015-03-19
3,0,3,4,2015-03-19
4,0,4,3,2015-03-19


# Read Business Data Set

In [7]:
#df_bus = pd.read_json('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/business.json', lines=True)

# Merge Business and Review dataset

In [8]:
#res_mer = pd.merge(df_bus,df_reviews, on='business_id')

In [9]:
#res_mer.head()

# Utility Matrix Creation

In [4]:
df = res_mer.pivot(index = 'user_id', columns ='business_id', values='rating')

In [5]:
rating_mat = df.as_matrix().astype(np.float)

# Centralizing each row vector to mean zero

In [12]:
# sum =0.0
# count =0
# for i in range(len(mat)):
#     sum = 0.0
#     count =0
#     for j in range(len(mat[0])):
#         if(math.isnan(mat[i][j])):
#             continue
#         sum += mat[i][j]
#         count += 1
#     avg = sum/count
#     for j in range(len(mat[0])):
#         if(math.isnan(mat[i][j])):
#             continue
#         mat[i][j] -= avg

In [6]:
arrMean = np.nanmean(rating_mat, axis = 1)

In [7]:
arrMean = arrMean[:, np.newaxis]

In [8]:
mat = rating_mat - arrMean

In [9]:
# Replace NaN values with zero in matrix
#ind = np.where(np.isnan(mat))
mat[np.isnan(mat)] = 0

In [10]:
print("Number of users:", len(mat))
print("Number of items:", len(mat[0]))

Number of users: 19771
Number of items: 48114


# Similarity Matrix

In [11]:
def sim_matrix(u, eps=1.0e-9):
    step1 = u.dot(u.T) + eps
    step2 = np.array([np.sqrt(np.diagonal(step1))])
    return (step1 / step2 / step2.T)

# Top K Similar Items

In [12]:
def top_k(arr, self_idx, k):
    sortedArr=arr.argsort()[::-1][:len(arr)]
    sortedArr=sortedArr[1:k+1]
    tk={}
    for i in sortedArr:
        if i != self_idx:
            tk[i]=arr[i]
    return tk

In [13]:
mat.shape[1]

48114

# Recommendation via Similar Users

In [14]:
def rec_via_users(m_utility, m_sim_users, user_idx, item_idx, k):
    dict_s = top_k(m_sim_users[user_idx], user_idx, k)
    s = 0.00
    c = 0.00
    for k,v in dict_s.items():
        if item_idx > m_utility.shape[1]:
            return 0
        if m_utility[k][item_idx] == 0:
            continue
        s += (m_utility[k][item_idx] * v)
        c += v
    if c == 0:
        return 0
    return s/c

In [ ]:
# # Finding user-index with atleast 5 ratings for items
# i = 0
# for i in range(len(mat)):
#     c = 0
#     for j in range(len(mat[0])):
#         if mat[i][j] > 0:
#             c = c+1
#     if c >4:
#         break
# print(i)

In [15]:
#u1 = np.array([[2, 0, 0, 4, 4], [5, 5, 5, 3, 3], [2, 4, 2, 1, 2]])
#Expected Rating of item number 4 for user number 3939
#print(rec_via_users(mat, sim_matrix(mat), 3939, 4, 10))
sim_mat_user = sim_matrix(mat)

MemoryError: 

In [ ]:
print(rec_via_users(mat,sim_mat_user,23,4,10))

In [ ]:
res_mer_test = pd.read_csv('D:\desk_2104\CS Sem4\CS6220\Project\dataset\yelp.test.rating', sep ='\t')

In [ ]:
mat.shape

In [63]:
df_test = res_mer_test.pivot(index = 'user_id', columns ='business_id', values='rating')
df_test.head()

business_id,5,7,8,9,14,15,17,27,36,38,...,48191,48200,48214,48221,48253,48297,48312,48320,48343,48344
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
mat_test = df_test.as_matrix().astype(np.float)

In [68]:
arrMean = np.nanmean(mat_test, axis = 1)
arrMean = arrMean[:,np.newaxis]
mat_test = mat_test - arrMean

In [70]:
mat_test[np.isnan(mat_test)] = 0

In [71]:
print("Number of users:", len(mat_test))
print("Number of items:", len(mat_test[0]))

Number of users: 19771
Number of items: 10841


In [93]:
pred_ratings = []
for index, row in res_mer_test.iterrows():
    pred = rec_via_users(mat, sim_mat_user, row.user_id, row.business_id, 10)
    pred_ratings.append(pred)

In [95]:
pred_ratings

[0,
 0,
 0,
 0,
 0,
 0,
 0.20833333333333304,
 0,
 0,
 0.10076613317811084,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0941176470588234,
 0,
 0,
 0,
 0,
 -0.26315789473684248,
 0,
 0.18518518518518517,
 0,
 0,
 -2.5769230769230771,
 0,
 0,
 0,
 -2.7599999999999998,
 0.59783135385380914,
 0,
 0,
 0,
 0.93103448275862066,
 -0.041666666666666963,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.35849056603773599,
 0,
 0,
 0,
 0,
 0,
 0,
 1.209090909090909,
 0,
 0,
 0,
 -1.1461175028635822,
 0,
 0.76190476190476186,
 0,
 0,
 0,
 0,
 0.55223880597014929,
 0,
 0,
 0,
 0,
 0,
 0,
 1.1481481481481484,
 0,
 0,
 0,
 0,
 0,
 0,
 0.65189873417721522,
 0,
 0,
 0,
 0,
 0,
 0.22988505747126436,
 0,
 0,
 0,
 0,
 0,
 0.53992094337207353,
 -0.78620689655172415,
 0,
 0,
 0,
 0,
 0,
 0.38,
 0,
 0,
 1.1384615384615384,
 0,
 0,
 0.1844511885529633,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.82894736842105265,
 0,
 0,
 -0.14623521353222929,
 -0.56097560975609773,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -0.239

In [98]:
res_mer_test

,user_id,business_id,rating,date
0,0,43,5,2016-04-27
1,1,72,5,2016-01-30
2,2,127,4,2015-07-18
3,3,151,5,2017-04-12
4,4,411,4,2017-05-01
5,5,437,4,2017-06-18
6,6,514,5,2017-07-23
7,7,43,5,2015-11-22
8,8,544,5,2017-07-20
9,9,401,5,2016-08-14


# SVD Recommendation

In [74]:
random.seed(12345)

def mse_utility(u1, u2):
    return mse(u1[u1.nonzero()].flatten(), u2[u2.nonzero()].flatten())

def svd_projection(m_utility, num_dims):
    U,S,VT = svds(m_utility, num_dims)
    SD = np.diag(S)
    return np.dot(np.dot(U,SD),VT)

def recs_via_svd(m_utility, num_dims, test_n):
    test = random.sample(range(m_utility.shape[0]), test_n)    
    svd_utility = svd_projection(m_utility, num_dims)
    
    true = []
    pred = []
    for user_idx in test:
        for item_idx in range(m_utility.shape[1]):
            if m_utility[user_idx][item_idx] != 0:
                true.append(m_utility[user_idx][item_idx])
                p = round(svd_utility[user_idx][item_idx])
                
                if p != 0:
                    pred.append(p)
                else:
                    pred.append(1.0e-9)

    return mse_utility(np.array([true], dtype=np.float64), np.array([pred], dtype=np.float64))



# Mean Square Error for rating prediction by SVD
print(recs_via_svd(mat, 200, 9500))

0.160562208905


# End of SVD #

In [34]:
# List of indices of similar users
similar_users = []
similarity = []
def cosine_similarity(user):
    for i in range(len(mat)):
        cs = 1 - dist.cosine(mat[i],user)
        if cs > 0:
            similarity.append(cs)
            similar_users.append(i)

In [35]:
#calling method
#Let new user be mat[320]
cosine_similarity(mat[320])

/home/gautam/.local/lib/python3.5/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [27]:
len(similar_users)

200

In [47]:
##Item Rating prediction for user
###INCORRECT - need to check the formula
item_ratings = np.zeros((len(mat[0])))
for i in range(len(similar_users)):
    u = mat[similar_users[i]]
    for j in range(len(u)):
        if(math.isnan(u[j])):
            continue
        else:
            item_ratings[j] += similarity[i] * u[j]

In [49]:
##Items recommended to user based on rating prediction
c = 0
for i in range(len(item_ratings)):
    if item_ratings[i]>0:
        c = c +1
print("count is:", c)

count is: 131


In [50]:
###Item Recommendation to users based on similar_users####
ind = similar_users[1]
recommend_items_indices = []

for i in range(len(similar_users)):
    for k,v in enumerate(mat[similar_users[i]]):
        if v>0:
            recommend_items_indices.append(k)

##Removing indices which are already used by user
for k,v in enumerate(mat[320]):
    if v > 0:
        recommend_items_indices.remove(k)
            
#Recommended items is
recommended_items = []

for i in range(len(recommend_items_indices)):
    recommended_items.append(items[recommend_items_indices[i]])


In [51]:
## recommended_items - user_items
len(recommended_items)

531

In [30]:
df_bus[df_bus['business_id'] == recommended_items[0]]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
33553,"3015 S Valley View Blvd, Ste 100",{'BusinessAcceptsCreditCards': True},MPGrHMnDFzXCr-EtNSDh2g,"[Automotive, Oil Change Stations, Smog Check S...",Las Vegas,"{'Monday': '7:30-17:00', 'Thursday': '7:30-17:...",1,36.136304,-115.191512,Bob's Autodynamics,Westside,89102,15,4.0,NV
